# Neuronales Netz zur Realisierung einer DNF

## 1.Zuerst wird die DNF erstellt

In [ ]:
import numpy as np

n = 10 #Anzahl der Eingabevariablen
m = 5 #Anzahl der Monome


#Definition der Monome als Liste von Literalen
# 1 für wahr, -1 für falsch, 0 für nicht verwendet
dnf = np.array([
    [ 1, -1,  1,  0,  0,  0,  0,  0,  0,  0],  # Monom 1: x1 ∧ ¬x2 ∧ x3
    [-1,  1,  0,  1,  0,  0,  0,  0,  0,  0],  # Monom 2: ¬x1 ∧ x2 ∧ x4
    [ 1,  0,  0, -1,  1,  0,  0,  0,  0,  0],  # Monom 3: x1 ∧ ¬x4 ∧ x5
    [ 0, -1,  1,  0,  1,  0,  0,  0,  0,  0],  # Monom 4: ¬x2 ∧ x3 ∧ x5
    [-1,  0,  0,  1, -1,  1,  0,  0,  0,  0],  # Monom 5: ¬x1 ∧ x4 ∧ ¬x5 ∧ x6
])

dnf2 = np.array([
    [ 1, -1,  1,  0,  0,  0,  0,  0,  0,  0],  # Monom 1: x1 ∧ ¬x2 ∧ x3
    [-1,  1,  -1,  0,  0,  0,  0,  0,  0,  0],  # Monom 2: ¬x1 ∧ x2 ∧ x4
    [ 1,  1,  -1,  0,  0,  0,  0,  0,  0,  0],  # Monom 3: x1 ∧ ¬x4 ∧ x5
    [ -1, -1,  1,  0,  0,  0,  0,  0,  0,  0],  # Monom 4: ¬x2 ∧ x3 ∧ x5
    [1,  -1,  -1,  0, 0,  0,  0,  0,  0,  0],  # Monom 5: ¬x1 ∧ x4 ∧ ¬x5 ∧ x6
])

## 2. Gewichtsmatrix und Schwellenwerte berechnen

In [ ]:
def calculate_weights_and_thresholds(dnf_matrix):
   """  w = dnf_matrix.copy() #Gewichte für erste Schicht
    v = np.sum(np.abs(w), axis=1) #Schwellenwerte für jede Zeile
    W = np.ones((1, m))
    V = -(m - 1)
    print(f"w ={w}, v = {v},/n W = {W}/n V = {V}")
    return w, v, W, V """
   w = dnf_matrix.astype(float) 
   v = np.sum(np.abs(w), axis=1).astype(float)
   W = np.ones((1, m), dtype=float)
   V = float(-(m - 1))
   return w, v, W, V

## 3.Vorzeichenfunktion definieren

In [ ]:
def sgn(x):
    return np.where(x >= 0, 1, -1)

## 4.Vorwärtspropagation

In [ ]:
def forward(x, w, v, W, V):
    z = sgn(np.dot(w, x) - v)
    y = sgn(np.dot(W, z) - V)
    return y, z

In [ ]:
def train(x, p, w, v, W, V, eta):
    y, z = forward(x, w, v, W, V)

    # Ausgabe-Schicht (Output)
    delta_V = -eta * (p - y)
    delta_W = eta * (p - y) * z
    V += delta_V
    W += delta_W

    # Versteckte Schicht (Hidden Layer)
    for j in range(W.shape[1]):
        delta_vj = -eta * (p - y) * W[0, j]
        delta_wj = eta * W[0, j] * (p - y) * x
        v[j] += delta_vj
        w[j] += delta_wj

    return w, v, W, V, y

## 5. Tests mit Beispiel

In [ ]:
""" # Beispielinput, der falsch sein soll (DNF = -1)
x = np.array([-1, -1, 1, -1, 1, -1, -1, 1, 1, 1])
p = 1  # erwartete Zielausgabe für x

# Gewichte direkt aus DNF bestimmen
w, v, W, V = calculate_weights_and_thresholds(dnf2)

y, z = forward(x, w, v, W, V)


print(f"Output = {y}")
# Lernrate
eta = 0.1




 """

In [ ]:
# Trainingsdaten: Tupel (x, Zielwert p)
training_data = [
    (np.array([-1, -1,  1, -1,  1, -1, -1,  1, 1, 1]),  1),
    (np.array([ 1,  1, 1, -1,  1,  1, -1,  1, 1, 1]),  -1),
    (np.array([ 1, -1,  1,  0,  0,  0,  0,  0, 0, 0]),  1),
    (np.array([-1, -1, -1,  1, -1,  1,  0,  0, 0, 0]), -1),
    (np.array([ 1, -1, -1,  0,  0,  0,  0,  0, 0, 0]),  1),
]
eta = 0.1
# Initialisierung der Gewichte über DNF
#w, v, W, V = calculate_weights_and_thresholds(dnf2)

w = np.random.uniform(-1, 1, (m, n))
v = np.random.uniform(0, 2, m)
W = np.random.uniform(-1, 1, (1, m))
V = np.random.uniform(-1, 1)
# Training
for epoch in range(20):
    total_error = 0
    for x, p in training_data:
        w, v, W, V, y = train(x, p, w, v, W, V, eta)
        total_error += int(y != p)
    print(f"Epoche {epoch + 1}, Fehler: {total_error}")

# Testausgabe nach dem Training
print("\nTestergebnisse nach Training:")
for x, p in training_data:
    y, _ = forward(x, w, v, W, V)
    print(f"Eingabe: {x}, Ziel: {p}, Ausgabe: {y}")

## Kontrolle ohne Neuronales Netz


In [ ]:
def evaluate_dnf(dnf_matrix, x):
    """
    Evaluates the DNF using logical operators.
    
    Parameters:
        dnf_matrix (numpy.ndarray): The DNF matrix where each row represents a monomial.
        x (numpy.ndarray): The input vector to evaluate.
        
    Returns:
        int: 1 if the input satisfies the DNF, otherwise -1.
    """
    for monomial in dnf_matrix:
        monomial_result = True
        for literal, value in zip(monomial, x):
            if literal == 1:  # Literal is positive
                monomial_result = monomial_result and (value == 1)
            elif literal == -1:  # Literal is negated
                monomial_result = monomial_result and (value == -1)
        if monomial_result:
            return 1  # At least one monomial is satisfied
    return -1  # None of the monomials are satisfied

for elem in training_data:
    print(evaluate_dnf(dnf2, elem[0]))